# Nodes and Edges

<b>Intent:</b> Taking a look at some of the things that you may need to use Python for in creating your network graphs.

<b>Data from:</b> Gabriella's spreadsheet

In [57]:
# import packages
from datascience import *
import pandas as pd

### Importing CSVs
We are going to read in a csv that we downloaded from Google Sheets, using the `read_table` method from `datascience`. Below is a picture of what it originally looked like.

![](Google Sheets.png)

In [58]:
graffiti = Table().read_table('WellonsG Graffiti mochicas in the Huaca Cao Viejo, El Brujo - Sheet1.csv')
graffiti

Temple,Code,Location,Width,Dimensions,Description
E,E1,West wall of a wall located at the NE corner of the pyra ...,7.2 cm,1.5 mm,Fish head with appendages projecting from the mouth. The ...
E,E2,"North surface of the second column (from East to West), ...",18.5 cm,19 cm,Rhombic design with volutes at vertices; Inside you can ...
E,E3,"Ur s surface of the second column (from East to West), o ...",18.5 cm,46.5 cm,"Three stylized human faces, one of which wears a simple ..."
D,D1,"South surface of a column painted white, in the lateral ...",33 cm,115 cm,"Stylized character, ornithomorphic motif and other like ..."
D,D2,"West surface painted red of a column, in the lateral sec ...",36 cm,92 cm,Character surrounded by a set of lines (Fig. 13B)
D,D3,"North facing north of enclosure 8, in the Patio Cereal, ...",10.5 cm,"15,5 cm",Character with headdress profile looking west. The body ...
D,D4,"North facing of the enclosure 8, in the Ceremonial Court ...",9 cm,20.5 cm,"Four separate motifs: a trophy head facing east, which s ..."
D,D5,"South facing interior of Campus 5, southwestern sector o ...",3.7 cm,6 cm,"Incomplete profile head, with headdress and earmuff, fac ..."
D,D6,"South facing of Campus 5, southwestern sector of the Upp ...",2.5 cm,2.5 cm,In-complete human face (Fig. 17B)
D,D7,"South facing interior of Campus 5, southwestern sector o ...",7.5 cm,13 cm,Undetermined reason (Fig. 17C)


In [59]:
# selecting out the columns that we want to turn into our node / edge pairs
temple_and_code = graffiti.select('Temple', 'Code')
temple_and_code

Temple,Code
E,E1
E,E2
E,E3
D,D1
D,D2
D,D3
D,D4
D,D5
D,D6
D,D7


### Adding IDs

We want to add a unique numerical ID to each of our items so that we can make our network graph. In order to do that, we will use a dictionary to keep track of what ID we assign to each different element. We are going to then define functions for adding elements to our dictionary and for getting the values that we assign to our keys.

In [60]:
# we are going to count up from zero as we assign ID's to keys
next_id = 0
# initializing the dictionary that we will keep our key-value pairs in
dictionary = {}

# making a function for adding keys
def add_to_dictionary(key):
    global next_id
    if key not in dictionary.keys():
        dictionary[key] = next_id
        next_id = next_id + 1

# a function to get back values from the dictionary
def get_id(key):
    return dictionary[key]

<b>Note:</b> \>>> (three greater than signs) means that that line is something we typed

#### <font color='blue'> Step by step of what we just defined</font>


```
>>> add_to_dictionary('E')
```

When we add 'dog' to our dictionary, we assign a unique ID to it, and store those values together. Think of this process as the computer remembering it like:

```
'dog' = 0
```
We then want to be able to get back an ID if we pass in a key:

```
>>> get_id('E')
0
```
This a repeatable process that will hold true as we continue to pass more things in.

```
>>> add_to_dictionary('D')
>>> add_to_dictionary('E1')
>>> add_to_dictionary('E2')
>>> get_id('E2')
3
>>> get_id('D')
1
```

In [61]:
# telling the computer, 'for each label in the labels
# of our temple_and_code table repeat this process':
for label in temple_and_code.labels:
    # apply the add_to_dictionary function to each value in the column 'label' of this table
    temple_and_code.apply(add_to_dictionary, label)

In [62]:
# creating columns for our node
nodes_and_edges = temple_and_code.with_columns([
        'Source', temple_and_code.apply(get_id, 'Temple'),
        'Edge', temple_and_code.apply(get_id, 'Code')
    ])
nodes_and_edges

Temple,Code,Source,Edge
E,E1,0,4
E,E2,0,5
E,E3,0,6
D,D1,1,7
D,D2,1,8
D,D3,1,9
D,D4,1,10
D,D5,1,11
D,D6,1,12
D,D7,1,13


### Getting Weights

We now have ID's for our table, but now we want weights for them that represent the strength of the relationship. There are many different ways to quantify a relationship, but we will do it based of frequency of appearance in our table. We will use the `group` function to let us know how many times a Temple-Code combo appears in our table.

In [63]:
weights = temple_and_code.group(['Temple', 'Code'])
weights

Temple,Code,count
A,A1,1
A,A10,1
A,A11,1
A,A12,1
A,A13,1
A,A14,1
A,A15,1
A,A16,1
A,A17,1
A,A18,1


We then will use the `pandas` function `merge` to join the two tables together so that we have the weights with our node and edge info. Notice that we need to convert our tables to `pandas` dataframes so that they will work with the function.

In [64]:
with_weights = pd.merge(nodes_and_edges.to_df(), weights.to_df(), how='right',on=['Temple', 'Code'])
# converting back to a datascience table
with_weights = Table().from_df(with_weights)
with_weights

Temple,Code,Source,Edge,count
E,E1,0,4,1
E,E2,0,5,1
E,E3,0,6,1
D,D1,1,7,1
D,D2,1,8,1
D,D3,1,9,1
D,D4,1,10,1
D,D5,1,11,1
D,D6,1,12,1
D,D7,1,13,1


In [65]:
# relabeling a column
with_weights.relabel('count', 'Weight')

Temple,Code,Source,Edge,Weight
E,E1,0,4,1
E,E2,0,5,1
E,E3,0,6,1
D,D1,1,7,1
D,D2,1,8,1
D,D3,1,9,1
D,D4,1,10,1
D,D5,1,11,1
D,D6,1,12,1
D,D7,1,13,1


In [66]:
# to save as a csv
#with_weights.to_csv('temple_nodes_edges.csv')